In [1]:
from pynq import Overlay
overlay = Overlay("/home/ubuntu/workspace/pynq_bitfiles/2-19/MatMul_SA5.bit")
overlay.download()

In [2]:
from pynq import Clocks
print(f"current IP PL clock hz: {Clocks.fclk0_mhz}Mhz")


current IP PL clock hz: 99.999Mhz


In [3]:
overlay.ip_dict.keys()

dict_keys(['mmult_accel_0', 'zynq_ultra_ps_e_0'])

In [4]:
ip_info = overlay.ip_dict['mmult_accel_0']

# Extract relevant fields
phys_addr = ip_info['phys_addr']
addr_range = ip_info['addr_range']

# Print in 8-bit hex format
print(f"'phys_addr': 0x{phys_addr:08X}, 'addr_range': 0x{addr_range:08X}")

'phys_addr': 0xB0000000, 'addr_range': 0x00010000


In [5]:
accel_ip = overlay.mmult_accel_0  # or whatever the block name is
# Write to an offset (e.g., 0x10) if that’s your control register
accel_ip.write(0x10, 0xFEEDBEEF)
# Read back
val = accel_ip.read(0x10)
print(hex(val))

0xfeedbeef


In [6]:
print(accel_ip.register_map)

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  A_1 = Register(A=write-only),
  A_2 = Register(A=write-only),
  B_1 = Register(B=write-only),
  B_2 = Register(B=write-only),
  C_1 = Register(C=write-only),
  C_2 = Register(C=write-only),
  N = Register(N=write-only),
  K = Register(K=write-only),
  M = Register(M=write-only)
}


In [7]:
# from pynq import MMIO

# # phys_addr = 0xA0000000, addr_range = 0x10000
# ip_base_addr = 2684354560   # 或者直接写 0xA0000000
# ip_addr_range = 65536       # 或者 0x10000

# mmult_mmio = MMIO(ip_base_addr, ip_addr_range)

# # 现在就可以对寄存器进行读写了，例如：
# mmult_mmio.write(0x10, 1234)         # 往 offset=0x10 写入 1234
# val = mmult_mmio.read(0x10)          # 读回 offset=0x10
# print("Read value:", val)

In [8]:
import time

def call_fpga(A_buf, B_buf, C_buf, accel_ip, N, K, M):
    """
    Runs a 2D matrix multiplication on the FPGA accelerator:
      (N, K) x (K, M) => (N, M)

    A_buf, B_buf, C_buf are PYNQ buffers allocated with shape=(N,K), (K,M), (N,M).
    """
    print("calling fpga")
    
    # 1. Flush input buffers so data is written to memory
    A_buf.flush()
    B_buf.flush()

    # 2. Configure the accelerator
    accel_ip.register_map.A_1 = A_buf.physical_address & 0xFFFFFFFF
    accel_ip.register_map.A_2 = (A_buf.physical_address >> 32) & 0xFFFFFFFF
    accel_ip.register_map.B_1 = B_buf.physical_address & 0xFFFFFFFF
    accel_ip.register_map.B_2 = (B_buf.physical_address >> 32) & 0xFFFFFFFF
    accel_ip.register_map.C_1 = C_buf.physical_address & 0xFFFFFFFF
    accel_ip.register_map.C_2 = (C_buf.physical_address >> 32) & 0xFFFFFFFF
    accel_ip.register_map.N = N
    accel_ip.register_map.K = K
    accel_ip.register_map.M = M

    # 3. Start the accelerator
    accel_ip.register_map.CTRL.AP_START = 1

    # 4. Wait for finish
    while accel_ip.register_map.CTRL.AP_DONE == 0:
        pass

    # 5. Invalidate output buffer (so CPU sees fresh data)
    C_buf.invalidate()

    # Return no explicit result here since C_buf is updated in place



In [9]:
import numpy as np
import torch
from pynq import allocate

In [10]:
# 2) Prepare Buffers (Using a small example for demonstration)
# Suppose we want to multiply a [N, K] x [K, M] => [N, M]
# DistilBERT typical Q/K/V matmul: shape [batch*seq_len, 768] x [768, 768]
# For demonstration, let’s do a small random size. We’ll adapt logic for DistilBERT below.
N, K, M = 64, 768, 768
A_buf = allocate(shape=(N, K), dtype=np.int8, cacheable=False)
B_buf = allocate(shape=(K, M), dtype=np.int8, cacheable=False)
C_buf = allocate(shape=(N, M), dtype=np.int32, cacheable=False) # accelerator writes int32 results

# 3) Initialize Input Data (simulate random int8 values)
A_buf[:] = np.random.randint(-128, 127, size=(N,K), dtype=np.int8)
B_buf[:] = np.random.randint(-128, 127, size=(K,M), dtype=np.int8)
# A_buf[:] = np.zeros((N, K), dtype=np.int8)
# B_buf[:] = np.zeros((K, M), dtype=np.int8)

# flush caches so data is in DDR
A_buf.flush()
B_buf.flush()

# 4) Configure Accelerator Control Registers
# Set Input & Output Buffer Addresses to Accelerator
accel_ip.register_map.A_1 = A_buf.physical_address & 0xFFFFFFFF
accel_ip.register_map.A_2 = (A_buf.physical_address >> 32) & 0xFFFFFFFF
accel_ip.register_map.B_1 = B_buf.physical_address & 0xFFFFFFFF
accel_ip.register_map.B_2 = (B_buf.physical_address >> 32) & 0xFFFFFFFF
accel_ip.register_map.C_1 = C_buf.physical_address & 0xFFFFFFFF
accel_ip.register_map.C_2 = (C_buf.physical_address >> 32) & 0xFFFFFFFF
accel_ip.register_map.N = N
accel_ip.register_map.K = K
accel_ip.register_map.M = M

In [11]:
print(f"'A_buf phys_addr': 0x{A_buf.physical_address:08X}")
print(f"'B_bufphys_addr': 0x{B_buf.physical_address:08X}")
print("IP Base Address: ", hex(accel_ip.mmio.base_addr))


'A_buf phys_addr': 0x375F0000
'B_bufphys_addr': 0x38000000
IP Base Address:  0xb0000000


In [25]:
import time

# ============================================
# 🚀 Step 3: BENCHMARK - FPGA ACCELERATOR
# ============================================

# FPGA Execution
start_data_in = time.time()
A_buf.flush()
B_buf.flush()
end_data_in = time.time()

start_fpga = time.time()
# accel_ip.register_map.CTRL.AP_START = 1
# while accel_ip.register_map.CTRL.AP_DONE == 0:
#     pass
call_fpga(A_buf, B_buf, C_buf, accel_ip, N, K, M)
end_fpga = time.time()

start_data_out = time.time()
C_buf.invalidate()
result_fpga = C_buf[:, :].copy()
end_data_out = time.time()

calling fpga


In [26]:
# ===========================
# 💻 Step 4: CPU REFERENCE (NumPy & PyTorch)
# ===========================
start_cpu_numpy = time.time()
A_sw = A_buf.astype(np.int32)
B_sw = B_buf.astype(np.int32)
ref_numpy = A_sw @ B_sw
# ref_numpy = -1
end_cpu_numpy = time.time()

start_cpu_torch = time.time()
device = torch.device("cpu")
A_torch = torch.tensor(A_buf.astype(np.int32), device=device)
B_torch = torch.tensor(B_buf.astype(np.int32), device=device)
ref_torch = torch.matmul(A_torch, B_torch)
ref_torch_np = ref_torch.cpu().numpy()
end_cpu_torch = time.time()

In [27]:
# ============================
# 🧪 Step 5: ACCURACY CHECK
# ============================
diff_numpy = np.abs(ref_numpy - result_fpga)
max_err_numpy = np.max(diff_numpy)

diff_torch = np.abs(ref_torch_np - result_fpga)
max_err_torch = np.max(diff_torch)

# ============================
# 📊 Step 6: PERFORMANCE METRICS
# ============================
total_ops = 2 * N * K * M

acc_latency = end_fpga - start_fpga
total_hw_time = end_data_out - start_data_in
hw_throughput = (total_ops / acc_latency) / 1e9
hw_end_to_end = (total_ops / total_hw_time) / 1e9

sw_time_numpy = end_cpu_numpy - start_cpu_numpy
sw_throughput_numpy = (total_ops / sw_time_numpy) / 1e9
speedup_latency_numpy = sw_time_numpy / acc_latency
speedup_total_numpy = sw_time_numpy / total_hw_time

sw_time_torch = end_cpu_torch - start_cpu_torch
sw_throughput_torch = (total_ops / sw_time_torch) / 1e9
speedup_latency_torch = sw_time_torch / acc_latency
speedup_total_torch = sw_time_torch / total_hw_time

In [28]:
print(result_fpga)
print(ref_torch_np)

[[-145470   33986  256613 ... -257507  267542   41925]
 [  72113  319012   59307 ...   30549  -32042 -164951]
 [ 267984   73130  -54719 ...  -10391 -119638  196044]
 ...
 [-275360   53641  -80537 ...   49469    6966  -48150]
 [  93598 -200453   18880 ...  -49456 -135556  252617]
 [  16469 -174141  -89092 ...  241539   17946   54187]]
[[-145470   33986  256613 ... -257507  267542   41925]
 [  72113  319012   59307 ...   30549  -32042 -164951]
 [ 267984   73130  -54719 ...  -10391 -119638  196044]
 ...
 [-275360   53641  -80537 ...   49469    6966  -48150]
 [  93598 -200453   18880 ...  -49456 -135556  252617]
 [  16469 -174141  -89092 ...  241539   17946   54187]]


In [29]:
# Determine emoji for accuracy check
diff_numpy_emoji = "❌" if max_err_numpy != 0 else "✅"
diff_torch_emoji = "❌" if max_err_torch != 0 else "✅"

# Define speedup emoji function
def speedup_emoji(speedup):
    if speedup >= 25:
        return "🚀🚀🚀"
    elif speedup >= 10:
        return "🚀🚀"
    elif speedup >= 1:
        return "🚀"
    return "🐢"

# ================================
# 🎯 Step 7: PERFORMANCE SUMMARY
# ================================
print("\n🎯 Performance Comparison")
print(f"🔢 MatMul Shape: [{N} , {K}] x [{K} , {M}]")

print(f"{diff_numpy_emoji} Max difference (NumPy vs FPGA): {max_err_numpy}")
print(f"{diff_torch_emoji} Max difference (PyTorch vs FPGA): {max_err_torch}\n")

print("📊 Latency & Throughput")
print(f"🧮 NumPy  : ⏱️ {sw_time_numpy:.6f} sec |  ⚡ {sw_throughput_numpy:.2f} GFLOPs")
print(f"🔥 PyTorch: ⏱️ {sw_time_torch:.6f} sec  |  ⚡ {sw_throughput_torch:.2f} GFLOPs")
print(f"🚀 FPGA   : ⏱️ {acc_latency:.6f} sec  |  ⚡ {hw_throughput:.2f} GFLOPs\n")

print(f"⏱️ Total HW Time : ⏱️ {total_hw_time:.6f} sec  |  ⚡ {hw_end_to_end:.2f} GFLOPs\n")

print("🚀 FPGA Speedup")
print(f"{speedup_emoji(speedup_latency_numpy)} Speedup vs NumPy   : {speedup_latency_numpy:.2f}x (Latency) | {speedup_total_numpy:.2f}x (Total)")
print(f"{speedup_emoji(speedup_latency_torch)} Speedup vs PyTorch : {speedup_latency_torch:.2f}x (Latency) | {speedup_total_torch:.2f}x (Total)")

print("\n✅ Test Completed! 🎯")


🎯 Performance Comparison
🔢 MatMul Shape: [64 , 768] x [768 , 768]
✅ Max difference (NumPy vs FPGA): 0
✅ Max difference (PyTorch vs FPGA): 0

📊 Latency & Throughput
🧮 NumPy  : ⏱️ 1.229659 sec |  ⚡ 0.06 GFLOPs
🔥 PyTorch: ⏱️ 0.098479 sec  |  ⚡ 0.77 GFLOPs
🚀 FPGA   : ⏱️ 0.039595 sec  |  ⚡ 1.91 GFLOPs

⏱️ Total HW Time : ⏱️ 0.044839 sec  |  ⚡ 1.68 GFLOPs

🚀 FPGA Speedup
🚀🚀🚀 Speedup vs NumPy   : 31.06x (Latency) | 27.42x (Total)
🚀 Speedup vs PyTorch : 2.49x (Latency) | 2.20x (Total)

✅ Test Completed! 🎯


### @TODO: CPU + FPGA hybrid LLM Acceleration


## Example: Integrating with DistilBERT (PyTorch) for Q, K, V, Out Projection in MHA


In [36]:
import transformers
# Load the DistilBERT base-uncased model
distilbert_model = transformers.DistilBertModel.from_pretrained("distilbert-base-uncased")
distilbert_model.eval() # set inference mode

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [92]:
# # Suppose we have an input tokenized and loaded into a torch tensor
# tokenizer = transformers.DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# text = "The moonlight shimmered over the quiet ocean as waves gently kissed the sandy shore, while distant lanterns flickered in the cool evening breeze. A lone traveler wandered along the coastline, footsteps sinking softly into the damp sand, lost in thought. The rhythmic sound of the water mixed with the rustling palms, creating a nice"
# tokens = tokenizer(text, return_tensors="pt")

In [93]:
# import time
# # torch.set_num_threads(1)  # Set to 1 thread to avoid CPU parallelism
# # torch.backends.mkldnn.enabled = False  # Disable MKL-DNN optimizations
# # torch.backends.openmp.enabled = False  # Disable OpenMP (multi-threading for CPU)

# start_db_cpu = time.time()
# with torch.no_grad():
#     # we can run the model once fully on CPU, just to see the normal output
#     cpu_output = distilbert_model(**tokens)
# end_db_cpu = time.time()
# print("CPU DistilBERT last hidden state shape:", cpu_output.last_hidden_state.shape)
# print(f"cpu run time: {(end_db_cpu-start_db_cpu):.6f} sec")

In [81]:
from pynq import allocate
import numpy as np
import torch

def distilbert_fpga_qkv_projection(
    hidden_states,
    q_weight, k_weight, v_weight,
    q_bias, k_bias, v_bias,
    accel_ip
):
    """
    Offloads Q, K, V projection in a single pass each
    for an entire sequence (batch=1).
    """
    batch_size, seq_len, hidden_dim = hidden_states.shape  # e.g., [1, 64, 768]
    # Weight shapes: [hidden_dim, qkv_dim], e.g. [768, 768] if head_dim * num_heads = 768
    _, qkv_dim = q_weight.shape

    # We assume batch_size=1 for simplicity
    assert batch_size == 1, "This example handles only batch_size=1. Loop over batch otherwise."

    # Shape for A is (64, 768), B is (768, qkv_dim), C is (64, qkv_dim)
    N = seq_len
    K = hidden_dim
    M = qkv_dim

    # Convert Torch -> NumPy
    A_np = hidden_states[0].cpu().numpy().astype(np.int8)  # shape (64, 768)
    Qw_np = q_weight.cpu().numpy().astype(np.int8)        # shape (768, qkv_dim)
    Kw_np = k_weight.cpu().numpy().astype(np.int8)
    Vw_np = v_weight.cpu().numpy().astype(np.int8)

    # Allocate PYNQ buffers
    A_buf  = allocate(shape=(N, K), dtype=np.int8)
    Qw_buf = allocate(shape=(K, M), dtype=np.int8)
    Kw_buf = allocate(shape=(K, M), dtype=np.int8)
    Vw_buf = allocate(shape=(K, M), dtype=np.int8)
    CQ_buf = allocate(shape=(N, M), dtype=np.int32)  # output for Q
    CK_buf = allocate(shape=(N, M), dtype=np.int32)  # output for K
    CV_buf = allocate(shape=(N, M), dtype=np.int32)  # output for V

    # Copy data to buffers
    A_buf[:]  = A_np
    Qw_buf[:] = Qw_np
    Kw_buf[:] = Kw_np
    Vw_buf[:] = Vw_np

    # 1) Q = hidden_states × q_weight
    call_fpga(A_buf, Qw_buf, CQ_buf, accel_ip, N, K, M)
    # 2) K = hidden_states × k_weight
    call_fpga(A_buf, Kw_buf, CK_buf, accel_ip, N, K, M)
    # 3) V = hidden_states × v_weight
    call_fpga(A_buf, Vw_buf, CV_buf, accel_ip, N, K, M)

    # Convert results back to NumPy float
    Q_out_np = CQ_buf.copy().astype(np.float32)  # shape (64, qkv_dim)
    K_out_np = CK_buf.copy().astype(np.float32)
    V_out_np = CV_buf.copy().astype(np.float32)

    # Free PYNQ buffers (optional, but good practice)
    A_buf.freebuffer()
    Qw_buf.freebuffer()
    Kw_buf.freebuffer()
    Vw_buf.freebuffer()
    CQ_buf.freebuffer()
    CK_buf.freebuffer()
    CV_buf.freebuffer()

    # Add bias & convert to Torch
    # biases [qkv_dim], so shape (768,) or similar
    Q_out = torch.tensor(Q_out_np, device=hidden_states.device) + q_bias
    K_out = torch.tensor(K_out_np, device=hidden_states.device) + k_bias
    V_out = torch.tensor(V_out_np, device=hidden_states.device) + v_bias

    # Reshape back to (batch_size, seq_len, qkv_dim)
    Q_out = Q_out.unsqueeze(0)  # shape [1, 64, qkv_dim]
    K_out = K_out.unsqueeze(0)
    V_out = V_out.unsqueeze(0)

    return Q_out, K_out, V_out


In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock_FPGA(nn.Module):
    def __init__(self, original_layer, accel_ip):
        super().__init__()
        self.original_layer = original_layer
        self.accel_ip = accel_ip

        # Extract submodules from the DistilBERT layer
        self.attention = original_layer.attention
        self.sa_layer_norm = original_layer.sa_layer_norm
        self.ffn = original_layer.ffn
        self.output_layer_norm = original_layer.output_layer_norm

    def forward(self, hidden_states, attention_mask=None):
        # 1) Grab Q, K, V weights and biases
        q_weight = self.attention.q_lin.weight
        k_weight = self.attention.k_lin.weight
        v_weight = self.attention.v_lin.weight
        q_bias   = self.attention.q_lin.bias
        k_bias   = self.attention.k_lin.bias
        v_bias   = self.attention.v_lin.bias

        # 2) Offload Q, K, V in one call each
        Q, K, V = distilbert_fpga_qkv_projection(
            hidden_states,
            q_weight, k_weight, v_weight,
            q_bias, k_bias, v_bias,
            self.accel_ip
        )

        # 3) Forward through attention
        attn_output_tuple = self.attention(Q, K, V, mask=attention_mask)
        attn_output = attn_output_tuple[0] if isinstance(attn_output_tuple, tuple) else attn_output_tuple

        # 4) Residual connection & layer norm
        hidden_states = self.sa_layer_norm(attn_output + hidden_states)

        # 5) Feed-forward network
        ffn_output = self.ffn(hidden_states)
        hidden_states = self.output_layer_norm(ffn_output + hidden_states)

        return hidden_states


In [83]:
class DistilBert_FPGA(nn.Module):
    def __init__(self, original_model, accel_ip):
        super().__init__()
        self.embeddings = original_model.embeddings  # Keep embeddings unchanged
        self.transformer_layers = nn.ModuleList([
            TransformerBlock_FPGA(layer, accel_ip) for layer in original_model.transformer.layer
        ])
        self.model = original_model  # Keep reference for methods like `forward`

    def forward(self, input_ids, attention_mask=None):
        hidden_states = self.embeddings(input_ids)

        for layer in self.transformer_layers:
            hidden_states = layer(hidden_states)

        return hidden_states  # No classifier, returning last hidden states


In [111]:
import torch
from transformers import DistilBertTokenizer

# 1) Load or define your normal CPU-based DistilBERT (same weights as distilbert_fpga)
#    If you used DistilBertModel: 
original_model = DistilBertModel.from_pretrained("distilbert-base-uncased")
#    If you used DistilBertForSequenceClassification:
# original_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
#
# Assume you've already done:
distilbert_fpga = DistilBert_FPGA(original_model, accel_ip)

# 2) Create tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# 3) Sample input text
text = (
    "The moonlight shimmered over the ocean as waves gently kissed the sandy shore, while distant lanterns flickered in the cool evening breeze. A lone traveler wandered along the coastline, footsteps sinking softly into the damp sand, lost in thought. The rhythmic sound of the water mixed with the rustling palms, creating a nice"
)

# 4) Tokenize (CPU)
tokens = tokenizer(text, return_tensors="pt")

input_ids = tokens["input_ids"]
attention_mask = tokens["attention_mask"]  # optional, if returned

# 5) Run normal CPU inference
# Make sure original_model is on CPU
original_model.to("cpu")
original_model.eval()

with torch.no_grad():
    # Output can be either 'last hidden states' or 'logits', depending on your model
    # DistilBertModel returns a BaseModelOutput, we can access last_hidden_state
    outputs_cpu = original_model(input_ids, attention_mask=attention_mask)
    if isinstance(outputs_cpu, tuple):
        # e.g., DistilBertModel returns (last_hidden_state, )
        cpu_result = outputs_cpu[0]
    else:
        # e.g., DistilBertForSequenceClassification returns (logits, )
        cpu_result = outputs_cpu.logits if hasattr(outputs_cpu, "logits") else outputs_cpu

# 6) Run FPGA inference
# distilbert_fpga is our custom model
distilbert_fpga.eval()  # ensure evaluation mode
with torch.no_grad():
    fpga_result = distilbert_fpga(input_ids, attention_mask=attention_mask)
    # If it returns a tuple, handle accordingly
    if isinstance(fpga_result, tuple):
        fpga_result = fpga_result[0]
    elif hasattr(fpga_result, "logits"):
        fpga_result = fpga_result.logits

# 7) Compare results
# We'll compute some difference metrics, like mean absolute difference (MAD)
diff = (cpu_result.last_hidden_state - fpga_result).abs()
mean_abs_diff = diff.mean().item()
max_abs_diff = diff.max().item()

print("CPU Result Shape:  ", cpu_result.last_hidden_state.shape)
print("FPGA Result Shape: ", fpga_result.shape)
print(f"Mean Absolute Difference: {mean_abs_diff:.6f}")
print(f"Max Absolute Difference:  {max_abs_diff:.6f}")

# For a quick pass/fail check:
if mean_abs_diff < 1e-3:
    print("The CPU and FPGA outputs match closely!")
else:
    print("Outputs differ by more than 1e-3. Check your FPGA calculations or quantization settings.")


calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
calling fpga
CPU Result Shape:   torch.Size([1, 63, 768])
FPGA Result Shape:  torch.Size([1, 63, 768])
Mean Absolute Difference: 0.294033
Max Absolute Difference:  9.816427
Outputs differ by more than 1e-3. Check your FPGA calculations or quantization settings.


In [112]:
cpu_result.last_hidden_state

tensor([[[-0.0687, -0.0838, -0.1465,  ...,  0.1789,  0.3295,  0.2308],
         [ 0.1156,  0.2173, -0.5855,  ...,  0.5756,  0.6235, -0.0118],
         [ 0.2028,  0.2943,  0.0740,  ...,  0.0933,  0.1457,  0.0222],
         ...,
         [-0.3366, -0.1055, -0.0118,  ..., -0.1256,  0.0316,  0.6437],
         [-0.3558, -0.5757,  0.1040,  ...,  0.2259,  0.0016, -0.1222],
         [-0.0130,  0.0660, -0.1378,  ...,  0.4973,  0.1388, -0.4929]]])

In [113]:
fpga_result

tensor([[[-0.0458,  0.0643,  0.1332,  ...,  0.0055,  0.1163,  0.1951],
         [-0.3400, -0.0190, -0.4053,  ...,  0.4973,  0.5385, -0.6715],
         [ 0.1272,  0.1808,  0.2804,  ..., -0.3034,  0.2214, -0.2878],
         ...,
         [-0.6369,  0.3454,  0.1677,  ..., -0.1749,  0.0853,  0.3318],
         [ 0.2345,  0.2810,  0.3364,  ..., -0.1767, -0.2867,  0.0592],
         [-0.0959,  0.0636,  0.1333,  ...,  0.6422,  0.0584,  0.2774]]])